<div style="background-color:teal; color:white; padding:10px; font-size:20px">
NB Setup

#### Installs

In [2]:
!conda update -n base -c defaults conda -y

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::bokeh==1.4.0=py37_0
  - defaults/noarch::dask==2.11.0=py_0
  - defaults/linux-64::distributed==2.11.0=py37_0
  - defaults/linux-64::spyder==4.0.1=py37_0
  - defaults/linux-64::watchdog==0.10.2=py37_0
failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::bokeh==1.4.0=py37_0
  - defaults/noarch::dask==2.

In [3]:
!conda install pytorch torchvision -c pytorch -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ffmpeg-4.2.2               |       h20bf706_0        59.6 MB
    flit-core-3.6.0            |     pyhd3eb1b0_0          42 KB
    gnutls-3.6.15              |       he1e5248_0         1.0 MB
    lame-3.100                 |       h7b6447c_0         323 KB
    libidn2-2.3.4              |       h5eee18b_0         146 KB
    libopus-1

In [4]:
!pip install boto3

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.8.3 requires ruamel-yaml, which is not installed.
awscli 1.29.14 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.31.14 which is incompatible.


#### Imports

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import boto3
import os

#### S3 Connection

In [7]:
s3 = boto3.client("s3")# Create a session using your AWS credentials
response = s3.list_buckets()# List all bucket names
buckets = [bucket["Name"] for bucket in response["Buckets"]]# Convert to list

# Print out the bucket list
print("Bucket List: %s" % buckets)

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Bucket List: ['behavior-project', 'pytorch-aws-train-deploy-project', 'sagemaker-us-east-1-717145514721']


<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Download Data (Batch Format)

In [8]:
# Download the CIFAR-10 dataset
download_transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=download_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=download_transform)

Files already downloaded and verified
Files already downloaded and verified


<div style="background-color:teal; color:white; padding:10px; font-size:20px">
Convert to Folder Format & upload to s3

In [ ]:
# Define function to save images in foldered format
def save_to_foldered_format(dataset, root_dir):
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
    for i, (image, label) in enumerate(dataset):
        label_dir = os.path.join(root_dir, dataset.classes[label])
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)
        image_path = os.path.join(label_dir, f"{i}.png")
        torchvision.utils.save_image(image, image_path)

# Save the datasets to the foldered format
save_to_foldered_format(trainset, './data/cifar10_foldered/train')
save_to_foldered_format(testset, './data/cifar10_foldered/val')

Upload to s3

In [ ]:
from tqdm import tqdm

def upload_dir_to_s3(bucket_name, source_directory, target_directory=""):
    s3 = boto3.client('s3')
    
    # Get the total number of files for tqdm
    total_files = sum([len(files) for subdir, _, files in os.walk(source_directory)])
    
    progress_bar = tqdm(total=total_files, desc="Uploading", unit="file")
    
    for subdir, _, files in os.walk(source_directory):
        for file in files:
            full_path = os.path.join(subdir, file)
            with open(full_path, 'rb') as data:
                relative_path = subdir.replace(source_directory, '').lstrip('/')
                key = os.path.join(target_directory, relative_path, file)
                s3.upload_fileobj(data, bucket_name, key)
                progress_bar.update(1)  # Update tqdm progress bar

    progress_bar.close()  # Close the tqdm progress bar once uploading is done

In [ ]:
# Set Paths
bucket_name = 'pytorch-aws-train-deploy-project'

train_source_directory = "/root/projects/pytorch_train_deploy/data/cifar10_foldered/train"
train_target_directory = "cifar-dataset/foldered-dataset/train"

val_source_directory = "/root/projects/pytorch_train_deploy/data/cifar10_foldered/val"
val_target_directory = "cifar-dataset/foldered-dataset/val"

# Upload
upload_dir_to_s3(bucket_name, train_source_directory, train_target_directory)
upload_dir_to_s3(bucket_name, val_source_directory, val_target_directory)

<div style="background-color:teal; color:white; padding:10px; font-size:20px">
(Subset) Convert to Foldered and Upload to s3

In [ ]:
# Define function to save images in foldered format
def save_to_foldered_format(dataset, root_dir, fraction=1.0):
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)

    num_images_to_save = int(len(dataset) * fraction)  # Compute the number of images to save
    
    for i, (image, label) in enumerate(dataset):
        if i >= num_images_to_save:
            break  # Stop after saving desired number of images

        label_dir = os.path.join(root_dir, dataset.classes[label])
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)

        image_path = os.path.join(label_dir, f"{i}.png")
        torchvision.utils.save_image(image, image_path)

In [ ]:
from tqdm import tqdm

def upload_dir_to_s3(bucket_name, source_directory, target_directory=""):
    s3 = boto3.client('s3')
    
    # Get the total number of files for tqdm
    total_files = sum([len(files) for subdir, _, files in os.walk(source_directory)])
    
    progress_bar = tqdm(total=total_files, desc="Uploading", unit="file")
    
    for subdir, _, files in os.walk(source_directory):
        for file in files:
            full_path = os.path.join(subdir, file)
            with open(full_path, 'rb') as data:
                relative_path = subdir.replace(source_directory, '').lstrip('/')
                key = os.path.join(target_directory, relative_path, file)
                s3.upload_fileobj(data, bucket_name, key)
                progress_bar.update(1)  # Update tqdm progress bar

    progress_bar.close()  # Close the tqdm progress bar once uploading is done

In [ ]:
# Upload the subset data to S3
bucket_name = 'pytorch-aws-train-deploy-project'

train_source_directory = "/root/Experiments_tut/toy_pytorch_project/data/cifar10_foldered_subset/train"
train_target_directory = "cifar-dataset/foldered-dataset-subset/train"

val_source_directory = "/root/Experiments_tut/toy_pytorch_project/data/cifar10_foldered_subset/val"
val_target_directory = "cifar-dataset/foldered-dataset-subset/val"

upload_dir_to_s3(bucket_name,train_source_directory, train_target_directory)
upload_dir_to_s3(bucket_name,val_source_directory, val_target_directory)